In [1]:
!pip install -q speechbrain
!pip install -q pyannote.audio

In [2]:
import warnings

warnings.filterwarnings("ignore", message="The OPUS subtype is unknown to TorchAudio.*")

import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

import os
import shutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device - {device}")

device - cuda


In [3]:
audio_dir = '/kaggle/input/temih-raw-audio-from-tg/'
# single_speeker_dif = 

In [4]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")

# instantiate the pipeline
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=hf_token)

pipeline.to(torch.device(device))

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [6]:
processed_count = 0
num_to_process = 1000

single_speaker_dir = '/kaggle/working/single_speaker'
os.makedirs(single_speaker_dir, exist_ok=True)

for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        try:
            if processed_count >= num_to_process:
                break
            audio_path = os.path.join(audio_dir, file_name)
            diarization = pipeline(audio_path)
            num_speakers = len(diarization.labels())
            if num_speakers == 1:
#                 print(f"\nfilename - {file_name}, num of speakers - {num_speakers}")
#                 for turn, _, speaker in diarization.itertracks(yield_label=True):
#                     start_time = turn.start
#                     end_time = turn.end
#                     print(f"\tSpeaker {speaker}: {start_time} - {end_time}")
                
                # Copy the single-speaker audio file to the new directory
                shutil.copy(audio_path, os.path.join(single_speaker_dir, file_name))
                print(f"Copied {file_name} to single_speaker directory")
            
            processed_count += 1
        except Exception as e:
            print(f"\nError processing {file_name}: {str(e)}")

print(f"\nTotal files processed: {processed_count}")
print(f"Single-speaker files saved in: {single_speaker_dir}")

	Copied message_63076.wav to single_speaker directory
	Copied message_310128.wav to single_speaker directory
	Copied message_62145.wav to single_speaker directory
	Copied message_72386.wav to single_speaker directory
	Copied message_74844.wav to single_speaker directory
	Copied message_87658.wav to single_speaker directory
	Copied message_53443.wav to single_speaker directory
	Copied message_52788.wav to single_speaker directory
	Copied message_53041.wav to single_speaker directory
	Copied message_63091.wav to single_speaker directory
	Copied message_85208.wav to single_speaker directory
	Copied message_62119.wav to single_speaker directory
	Copied message_72362.wav to single_speaker directory
	Copied message_65928.wav to single_speaker directory
	Copied message_78967.wav to single_speaker directory
	Copied message_66962.wav to single_speaker directory
	Copied message_54280.wav to single_speaker directory

Error processing message_64260.wav: Sizes of tensors must match except in dimens

In [8]:
from speechbrain.inference.speaker import SpeakerRecognition
import torchaudio

verification = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="./pretrained_ecapa",
    run_opts={"device": device}
)


In [21]:
embeddings = []
labels = []

for file_name in os.listdir(single_speaker_dir):
    if file_name.endswith('.wav'):
        try:
            file_path = os.path.join(single_speaker_dir, file_name)
            signal, fs = torchaudio.load(file_path)
            embedding = verification.encode_batch(signal)
            print(f"embedding - {embedding}")
            print(f"embedding.shape - {embedding.shape}")
            print(f"embedding[0, 0] - {embedding[0, 0]}")
            print(f"embedding[0, 0].shape - {embedding[0, 0].shape}")
            embeddings.append(embedding[0, 0].cpu().numpy())
            labels.append(file_name)
        except Exception as e:
            print(f"Error processing {file_name}: {str(e)}")
    break

embedding - tensor([[[ 2.3085e+01, -2.9057e+00, -1.3077e+01,  1.2054e+00,  1.5207e+01,
           1.4113e+01, -6.0674e+00,  3.6643e+01, -6.2358e+00,  3.3546e+00,
           1.8203e+01, -2.4042e+00,  8.9109e+00, -1.1671e+01, -5.6523e+00,
           5.7315e+00,  9.7818e-01,  3.3384e+01,  9.9278e+00,  1.0600e+01,
           4.4214e+01,  1.4832e+01, -3.4177e+01,  3.2961e+00,  2.4899e+01,
          -5.4729e+00, -1.4917e+00, -1.2814e+01,  1.9850e+01, -1.0325e+01,
          -8.9156e+00, -2.5183e+01, -2.0998e+00, -6.9414e+00,  1.6820e+01,
           6.2050e-02,  9.0115e+00,  6.6656e+00,  1.8870e+01,  7.1132e+01,
           2.4401e+01, -4.2474e+00,  2.1534e+01, -6.1734e+01,  6.8881e+00,
          -1.2127e+01,  1.3815e+01, -1.3351e+01,  4.7485e+01,  1.5039e+01,
           1.8235e+01, -4.3844e+00,  7.4586e+00,  1.8693e+01, -1.0000e+01,
          -1.6222e+01, -2.3370e+01,  1.1205e+01, -2.3261e+01,  1.7646e+01,
           4.3145e+01, -1.9242e+01, -3.1132e+01,  3.1201e+00, -9.3701e+00,
          -9.

In [ ]:
# # audio_dir = '/kaggle/input/temih-raw-audio-from-tg/'


In [ ]:
# import torch
# import torchaudio
# from speechbrain.inference.speaker import SpeakerRecognition
# import logging
# from sklearn.decomposition import PCA
# import os
# import numpy as np
# from pathlib import Path
# from pyannote.audio import Pipeline

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# verification = SpeakerRecognition.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="./pretrained_models/spkrec-ecapa-voxceleb",
#     run_opts={"device": device}
# )

In [ ]:
# !curl -O https://www.uclass.psychol.ucl.ac.uk/Release2/Conversation/AudioOnly/wav/F_0101_13y1m_1.wav
# !curl -O https://www.uclass.psychol.ucl.ac.uk/Release2/Conversation/AudioOnly/wav/M_0061_18y5m_1.wav

In [ ]:
# # audio_dir = '/kaggle/input/temih-raw-audio-from-tg/'
# audio_dir = '/kaggle/working/'

In [ ]:
# processed_count = 0
# for file_name in os.listdir(audio_dir):
#     if file_name.endswith('.wav'):
# #         if processed_count >= 100000:
# #             break
#         audio_path = os.path.join(audio_dir, file_name)
#         signal, fs = torchaudio.load(audio_path)
# #         print(f"signal.shape - {signal.shape}")
# #         print(f"fs - {fs}")
#         embedding = verification.encode_batch(signal)
#         if embedding.shape[2] > 1:
#             print(f"embedding.shape - {embedding.shape}; filename - {file_name}")
            
# #         print(f"embedding - {embedding}")
#         processed_count += 1